In [ ]:
import pandas as pd
import numpy as np
import random

random.seed(42)
np.random.seed(42)

def generate_hygiene_dataset(n_samples=5000, missing_rate=0.9):
    brands = [
        "Head&Shoulders", "Pantene", "Dove", "Nivea", "Garnier", "L'Oréal", "Clear",
        "Colgate", "Sensodyne", "Signal", "Blend-a-med", "Rexona", "Axe", "Fa",
        "Old Spice", "Palmolive", "Syoss", "Elseve", "Gliss Kur", "Timotei"
    ]
    product_types = {
        "Шампунь": ["шампунь", "шампунь против перхоти", "шампунь для мужчин", "шампунь 3 в 1", "сухой шампунь"],
        "Гель для душа": ["гель для душа", "душевой гель", "гель-скраб для тела", "крем-гель для душа"],
        "Зубная паста": ["зубная паста", "отбеливающая паста", "паста для чувствительных зубов", "комплексный уход"],
        "Крем для рук": ["крем для рук", "питательный крем", "увлажняющий крем", "защитный крем"],
        "Бальзам для губ": ["бальзам для губ", "защитный бальзам", "увлажняющий бальзам", "оттеночный бальзам"],
        "Дезодорант": ["спрей-дезодорант", "шариковый дезодорант", "антиперспирант", "стик-дезодорант"], # New category
        "Мыло": ["твердое мыло", "жидкое мыло", "антибактериальное мыло", "мыло-скраб"] # New category
    }
    modifiers = [
        "с аргановым маслом", "с ментолом", "с экстрактом ромашки", "для ежедневного использования",
        "антибактериальный", "без сульфатов", "с коллагеном", "гипоаллергенный", "для чувствительной кожи", "с витаминами"
    ]

    data = []
    for _ in range(n_samples):
        cat = random.choice(list(product_types.keys()))
        name = f"{random.choice(brands)} {random.choice(product_types[cat])} {random.choice(modifiers)}"

        if cat in ["Шампунь", "Гель для душа", "Жидкое мыло"]:
            volume = random.choice([200, 250, 400, 500, 750, 1000])
            unit = "мл"
        elif cat in ["Зубная паста", "Крем для рук", "Бальзам для губ", "Твердое мыло"]:
            volume = random.choice([50, 75, 100, 150, 200])
            unit = "мл" if cat in ["Зубная паста", "Крем для рук", "Бальзам для губ"] else "г"
        elif cat == "Дезодорант":
            volume = random.choice([50, 75, 150])
            unit = "мл"
        else:
            volume = random.choice([100, 200, 300])
            unit = "мл"

        data.append({"Название": name.strip(), "Категория": cat, "Объем": f"{volume} {unit}"})

    df = pd.DataFrame(data)
    mask = np.random.rand(len(df)) < (1 - missing_rate)
    df.loc[~mask, "Категория"] = None
    return df

df = generate_hygiene_dataset()
df.to_csv("hygiene_products_expanded.csv", index=False)
print("Expanded dataset saved to hygiene_products_expanded.csv")
print(df.head())

Expanded dataset saved to hygiene_products_expanded.csv
                                            Название Категория   Объем
0           Nivea спрей-дезодорант антибактериальный      None   50 мл
1      Garnier гель для душа для чувствительной кожи      None  200 мл
2              Fa бальзам для губ с аргановым маслом      None   50 мл
3      Colgate гель для душа для чувствительной кожи      None  250 мл
4  Elseve стик-дезодорант для ежедневного использ...      None   75 мл


In [ ]:
df['Название'].value_counts(), df['Категория'].value_counts()

(Название
 Timotei крем-гель для душа для ежедневного использования      5
 Dove стик-дезодорант с экстрактом ромашки                     5
 Fa увлажняющий бальзам с аргановым маслом                     5
 Axe комплексный уход с витаминами                             5
 Dove душевой гель с коллагеном                                5
                                                              ..
 Head&Shoulders шампунь 3 в 1 для ежедневного использования    1
 Colgate питательный крем без сульфатов                        1
 Head&Shoulders шампунь для мужчин гипоаллергенный             1
 Garnier сухой шампунь с коллагеном                            1
 Signal крем-гель для душа для чувствительной кожи             1
 Name: count, Length: 3354, dtype: int64,
 Категория
 Гель для душа      86
 Мыло               82
 Дезодорант         81
 Зубная паста       79
 Бальзам для губ    73
 Крем для рук       66
 Шампунь            58
 Name: count, dtype: int64)

In [ ]:
df['Категория'].isna().sum()

np.int64(4475)

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

df = pd.read_csv("hygiene_products.csv")
train = df[df["Категория"].notna()]
test = df[df["Категория"].isna()]

def get_true_cat(name):
    if "шампунь" in name.lower(): return "Шампунь"
    elif "гель для душа" in name.lower() or "душевой гель" in name.lower(): return "Гель для душа"
    elif "зубная паста" in name.lower() or "паста" in name.lower(): return "Зубная паста"
    elif "крем для рук" in name.lower(): return "Крем для рук"
    elif "бальзам для губ" in name.lower(): return "Бальзам для губ"
    else: return "Unknown"

test["true"] = test["Название"].apply(get_true_cat)
test = test[test["true"] != "Unknown"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
X_train = model.encode(train["Название"].tolist())
X_test = model.encode(test["Название"].tolist())

knn = KNeighborsClassifier(n_neighbors=5, metric='cosine')
knn.fit(X_train, train["Категория"])
preds = knn.predict(X_test)

f1 = f1_score(test["true"], preds, average='macro')
print(f"Embedding + k-NN → F1-macro: {f1:.3f}")

/tmp/ipython-input-652601339.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["true"] = test["Название"].apply(get_true_cat)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding + k-NN → F1-macro: 0.956


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df_expanded = pd.read_csv("hygiene_products_expanded.csv")
print("Expanded dataset loaded. Head:\n", df_expanded.head())

df_labeled = df_expanded[df_expanded["Категория"].notna()].copy()
print("\nDataFrame after filtering for labeled categories. Head:\n", df_labeled.head())
print("Number of labeled samples:", len(df_labeled))

train_df, val_df = train_test_split(df_labeled, test_size=0.2, random_state=42)

print("\nTraining set size:", len(train_df))
print("Validation set size:", len(val_df))
print("Training set head:\n", train_df.head())
print("Validation set head:\n", val_df.head())

Expanded dataset loaded. Head:
                                             Название Категория   Объем
0           Nivea спрей-дезодорант антибактериальный       NaN   50 мл
1      Garnier гель для душа для чувствительной кожи       NaN  200 мл
2              Fa бальзам для губ с аргановым маслом       NaN   50 мл
3      Colgate гель для душа для чувствительной кожи       NaN  250 мл
4  Elseve стик-дезодорант для ежедневного использ...       NaN   75 мл

DataFrame after filtering for labeled categories. Head:
                                              Название        Категория   Объем
6   Sensodyne отбеливающая паста для ежедневного и...     Зубная паста  100 мл
10            Signal увлажняющий бальзам с витаминами  Бальзам для губ   75 мл
29            Signal жидкое мыло с экстрактом ромашки             Мыло  200 мл
32                  Pantene душевой гель с витаминами    Гель для душа  200 мл
37     Rexona стик-дезодорант для чувствительной кожи       Дезодорант   75 мл
Number of 

## Выбор и загрузка базовой модели


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

labels = df_labeled["Категория"].unique().tolist()
num_labels = len(labels)
print(f"Unique labels: {labels}")
print(f"Number of labels: {num_labels}")

model_name = "bert-base-multilingual-cased"
print(f"\nSelected model: {model_name}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Tokenizer loaded successfully.")

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
print("Model loaded successfully.")

Unique labels: ['Зубная паста', 'Бальзам для губ', 'Мыло', 'Гель для душа', 'Дезодорант', 'Крем для рук', 'Шампунь']
Number of labels: 7

Selected model: bert-base-multilingual-cased


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Tokenizer loaded successfully.


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully.


In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

label_to_id = {label: i for i, label in enumerate(labels)}
id_to_label = {i: label for i, label in enumerate(labels)}
print(f"Label to ID mapping: {label_to_id}")

def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["Название"], truncation=True, padding="max_length")
    tokenized_inputs["labels"] = [label_to_id[cat] for cat in examples["Категория"]]
    return tokenized_inputs

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

print("\nTokenized training dataset head:")
print(tokenized_train_dataset[0])
print("\nTokenized validation dataset head:")
print(tokenized_val_dataset[0])
print("\nData preparation complete.")

Label to ID mapping: {'Зубная паста': 0, 'Бальзам для губ': 1, 'Мыло': 2, 'Гель для душа': 3, 'Дезодорант': 4, 'Крем для рук': 5, 'Шампунь': 6}


Map:   0%|          | 0/420 [00:00<?, ? examples/s]

Map:   0%|          | 0/105 [00:00<?, ? examples/s]


Tokenized training dataset head:
{'Название': 'Pantene душевой гель для ежедневного использования', 'Категория': 'Гель для душа', 'Объем': '250 мл', '__index_level_0__': 2023, 'input_ids': [101, 18661, 52423, 11576, 60229, 11292, 65100, 12118, 10520, 546, 15920, 10746, 10695, 47301, 52316, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True
)

def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=1)
    return {
        "f1_macro": f1_score(p.label_ids, predictions, average='macro'),
        "accuracy": accuracy_score(p.label_ids, predictions)
    }

print("Training arguments defined and compute_metrics function created.")

Training arguments defined and compute_metrics function created.


In [ ]:
from transformers import Trainer, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

print("Model training complete.")

/tmp/ipython-input-1403622663.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:wandb: You chose 'Create a W&B account'
wandb: Create an account here: https://wandb.ai/authorize?signup=true&ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ilya-s-2002 (ilya-s-2002-itmo-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Epoch,Training Loss,Validation Loss,F1 Macro,Accuracy
1,1.943900,1.929480,0.055347,0.142857
2,1.892600,1.776426,0.356077,0.419048
3,1.294800,0.977530,1.000000,1.000000


Model training complete.


## Оценка файн-тюнинговой модели

### Subtask:
Оценить производительность дообученной модели на валидационной выборке, используя метрику F1-score, чтобы убедиться в улучшении качества предсказаний.


**Reasoning**:
The model has been trained, so the next step is to evaluate its performance on the validation dataset using the `trainer.evaluate()` method to get the F1-score and other metrics.



In [ ]:
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

Epoch,Training Loss,Validation Loss,F1 Macro,Accuracy
0,No log,0.961483,1.000000,1.000000


Evaluation results: {'eval_loss': 0.9614825248718262, 'eval_f1_macro': 1.0, 'eval_accuracy': 1.0}


## Применение модели для предсказания пропущенных значений



In [ ]:
import pandas as pd
from datasets import Dataset

df_missing_categories = df_expanded[df_expanded["Категория"].isna()].copy()
print("DataFrame with missing categories created. Head:\n", df_missing_categories.head())
print("Number of samples with missing categories:", len(df_missing_categories))

DataFrame with missing categories created. Head:
                                             Название Категория   Объем
0           Nivea спрей-дезодорант антибактериальный       NaN   50 мл
1      Garnier гель для душа для чувствительной кожи       NaN  200 мл
2              Fa бальзам для губ с аргановым маслом       NaN   50 мл
3      Colgate гель для душа для чувствительной кожи       NaN  250 мл
4  Elseve стик-дезодорант для ежедневного использ...       NaN   75 мл
Number of samples with missing categories: 4475


In [ ]:
import numpy as np

missing_dataset = Dataset.from_pandas(df_missing_categories)
print("Missing categories dataset created.")

def tokenize_for_prediction_function(examples):
    tokenized_inputs = tokenizer(examples["Название"], truncation=True, padding="max_length")
    return tokenized_inputs

tokenized_missing_dataset = missing_dataset.map(tokenize_for_prediction_function, batched=True)
print("Missing categories dataset tokenized.")

tokenized_missing_dataset = tokenized_missing_dataset.remove_columns([col for col in tokenized_missing_dataset.column_names if col not in ['input_ids', 'attention_mask', 'token_type_ids']])

predictions_output = trainer.predict(tokenized_missing_dataset)
print("Predictions generated.")

predicted_ids = np.argmax(predictions_output.predictions, axis=1)
predicted_labels = [id_to_label[id_val] for id_val in predicted_ids]
print("Numerical predictions converted to text labels.")

df_expanded.loc[df_expanded["Категория"].isna(), "Категория"] = predicted_labels

print("Missing categories filled in df_expanded.")
print("Final df_expanded head with filled categories:")
print(df_expanded.head())
print("Number of remaining missing categories:", df_expanded["Категория"].isna().sum())

Missing categories dataset created.


Map:   0%|          | 0/4475 [00:00<?, ? examples/s]

Missing categories dataset tokenized.


Epoch,Training Loss,Validation Loss,F1 Macro,Accuracy
0,No log,0.961483,1.000000,1.000000


Predictions generated.
Numerical predictions converted to text labels.
Missing categories filled in df_expanded.
Final df_expanded head with filled categories:
                                            Название        Категория   Объем
0           Nivea спрей-дезодорант антибактериальный       Дезодорант   50 мл
1      Garnier гель для душа для чувствительной кожи    Гель для душа  200 мл
2              Fa бальзам для губ с аргановым маслом  Бальзам для губ   50 мл
3      Colgate гель для душа для чувствительной кожи    Гель для душа  250 мл
4  Elseve стик-дезодорант для ежедневного использ...       Дезодорант   75 мл
Number of remaining missing categories: 0
